<a href="https://colab.research.google.com/github/razzlestorm/DS-Unit-2-Kaggle-Challenge/blob/master/module3/Jerimiah_Willhite_assignment_kaggle_challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [17]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
Already up to date.


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')
print(train.shape, test.shape)

(59400, 41) (14358, 40)


In [0]:
# Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
#Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue Submit Predictions button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
#Commit your notebook to your fork of the GitHub repo.
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

In [0]:
import numpy as np


def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
target='status_group'
features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [0]:
#hand-picked categorical features, can comment or not
# categorical_features = ['public_meeting', 'management',
#                         'scheme_management', 'payment', 'quantity', 'extraction_type',
#                         'payment_type', 'waterpoint_type', 'extraction_type_class', 'water_quality',
#                         'source_class', 'source_type', 'basin', 'region', 'permit']

In [93]:
#let's make some pipelines!
import category_encoders as ce
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

param_distributions = {
    'randomforestclassifier__n_estimators': range(50,500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, None],
    'randomforestclassifier__max_features': uniform(0,1),
    'randomforestclassifier__min_samples_leaf': [1, 10, 100],
    'randomforestclassifier__criterion': ['gini', 'entropy'],
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions = param_distributions,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.5min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 18.3min finished


In [94]:
#Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html
print('Best hyperparameters', search.best_params_)
print('Cross-validation Accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 25, 'randomforestclassifier__max_features': 0.13234576637147177, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__n_estimators': 430}
Cross-validation Accuracy 0.8025462962962963


In [90]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,3,9,1,6,0,2,4,7,5,8
mean_fit_time,100.81,61.2493,124.413,18.9092,13.1659,26.5723,41.4356,29.2739,86.4726,57.4077
std_fit_time,1.92503,2.7387,2.94207,0.153829,0.134491,0.352111,0.250287,0.391981,0.419201,0.299406
mean_score_time,2.27726,0.827671,1.3656,0.623609,2.12849,0.833112,0.872489,0.544989,1.0706,0.775351
std_score_time,0.0383219,0.137475,0.027538,0.0045487,0.132811,0.0160928,0.0322734,0.00136623,0.0405277,0.0222582
param_randomforestclassifier__criterion,gini,entropy,gini,entropy,gini,entropy,gini,entropy,gini,gini
param_randomforestclassifier__max_depth,None,15,20,10,None,10,5,5,5,5
param_randomforestclassifier__max_features,0.406052,0.463177,0.900787,0.231687,0.00126179,0.236433,0.57573,0.543526,0.872327,0.889917
param_randomforestclassifier__min_samples_leaf,1,1,1,10,10,100,100,1,100,1
param_randomforestclassifier__n_estimators,351,155,209,120,479,208,324,157,450,285
params,"{'randomforestclassifier__criterion': 'gini', ...",{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...",{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...",{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...",{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...","{'randomforestclassifier__criterion': 'gini', ..."


In [55]:


from sklearn.model_selection import GridSearchCV
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42)
)

param_grid = {
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, None],
}



grid_search = GridSearchCV(
    pipeline,
    param_grid = param_grid,
    cv=6,
    scoring='accuracy',
    refit='AUC',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

grid_search.fit(X_train, y_train);

Fitting 6 folds for each of 6 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   50.7s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [56]:
print('Best hyperparameters', grid_search.best_params_)
print('Cross-validation Accuracy', grid_search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 15}
Cross-validation Accuracy 0.7916877104377105


In [60]:

param_grid2 = {
    'randomforestclassifier__min_samples_leaf': [1, 10, 100],
}

grid_search2 = GridSearchCV(
    pipeline,
    param_grid = param_grid2,
    cv=10,
    scoring='accuracy',
    refit='AUC',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

grid_search2.fit(X_train, y_train);

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [76]:
print('Best hyperparameters', grid_search2.best_params_)
print('Cross-validation Accuracy', grid_search2.best_score_)

Best hyperparameters {'randomforestclassifier__min_samples_leaf': 1}
Cross-validation Accuracy 0.7893308080808081


In [74]:

param_grid3 = {
    'randomforestclassifier__min_weight_fraction_leaf': [0.0, 0.1, 0.3, 0.5]
}



grid_search3 = GridSearchCV(
    pipeline,
    param_grid = param_grid3,
    cv=10,
    scoring='accuracy',
    refit='AUC',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

grid_search3.fit(X_train, y_train);


Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [75]:
print('Best hyperparameters', grid_search3.best_params_)
print('Cross-validation Accuracy', grid_search3.best_score_)

Best hyperparameters {'randomforestclassifier__min_weight_fraction_leaf': 0.0}
Cross-validation Accuracy 0.7893308080808081


In [64]:
param_grid = {
    'randomforestclassifier__max_depth': [5, 10, 15, 20, 25, None],
    'randomforestclassifier__min_weight_fraction_leaf': [0.0, 0.25, 0.5],
    'randomforestclassifier__min_samples_leaf': [1, 10, 100],
}



grid_search4 = GridSearchCV(
    pipeline,
    param_grid = param_grid,
    cv=3,
    scoring='accuracy',
    refit='AUC',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

grid_search4.fit(X_train, y_train);

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   17.2s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  2

In [65]:
print('Best hyperparameters', grid_search4.best_params_)
print('Cross-validation Accuracy', grid_search4.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_weight_fraction_leaf': 0.1}
Cross-validation Accuracy 0.6856481481481481


In [0]:
random_best = search.best_estimator_
random_index = search.best_index_
random_y_pred = best.predict(X_val)

In [96]:
search.score(X_val, y_val)

0.8132154882154882

In [71]:
grid_search4.score(X_val, y_val)

0.6824915824915825

In [100]:
from sklearn.ensemble import RandomForestClassifier


rm = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=430,
                           max_features=0.132346,
                           max_depth=20,
                           criterion='entropy',
                           min_samples_leaf=1,
                           random_state=42)
)

rm.fit(X_train, y_train)

print('Training accuracy: ', rm.score(X_train, y_train))
print('Validation accuracy: ', rm.score(X_val, y_val))

Training accuracy:  0.9705176767676768
Validation accuracy:  0.8127946127946128


In [0]:
y_pred = search.predict(X_test)
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-07.csv', index=False)
if in_colab:
  from google.colab import files
  files.download('submission-07.csv')